## Model for network of heat pump shown home

This code loads the required EPC/MCS data and generates a network for show homes with the given parameters. Gradio helps create an interactive interface for testing out different scenarios. 

In [20]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import gradio as gr

from show_homes import PROJECT_DIR, config
from show_homes.pipeline import show_homes_network
from show_homes.getters.show_home_data import get_show_home_data
from show_homes.config import config
from show_homes.utils import geo_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
df = get_show_home_data()

In [22]:
_ = show_homes_network.model_network(df, 'Any', True, 5, 1, 5, 6, 30, 'Orkney Islands', verbose=True)

Settings
************
Area: Orkney Islands
Property type: Any
Visitor ratio: 0.05
Host ratio: 0.01
Maximum visitors: 30
Maximum distance: 30
Same property type or not: True

(117, 3)
Before subsampling:
# Props without HPs: 2357
# Props with HPs: 1377

After subsampling:
# Props without HPs: 117
# Props with HPs: 13
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to /Users/juliasuter/Documents/repositories/show_homes/show_homes/analysis/maps/Generated_network_map_Any_True_0.01_0.05_30_30_Orkney_Islands.html!


In [25]:

def G(property_type, same_prop_type, 
                            visitor_ratio, host_ratio, 
                            v_max, n_open_days, d_max, local_auth, verbose=False):
    
    return show_homes_network.model_network(df, property_type, same_prop_type, 
                            visitor_ratio, host_ratio,
                            v_max, n_open_days, d_max, local_auth, verbose=verbose)
 
    
property_types = ['Flat', 'Semi-detached House', 'Detached House', 'Terraced House', 'Any']
local_authorities = [la for la in sorted(list(df['LOCAL_AUTHORITY_LABEL'].unique())) + ['GB'] if la != 'unknown']
    
demo = gr.Interface(
    fn=G,
    inputs=[gr.components.Radio(property_types, label='Property Type', value='Detached House'), 
            gr.components.Radio([True, False], label='Show home of same property', value=True), 
            gr.components.Slider(0, 100, value=5, step=1, label='Visitor ratio (%)'),
            gr.components.Slider(0, 100, value=1, step=1, label='Host ratio (%)'), 
            gr.components.Slider(1, 50, value=5, step=1, label='Max visitors'), 
            gr.components.Slider(1, 50, value=6, step=1, label='Number of slots/open days'), 
            gr.components.Slider(1, 50, value=35, step=1, label='Max distance'),
            gr.components.Dropdown(local_authorities, value='Orkney Islands', label='Local authorities')],
    outputs=["text", "html"],
    title='Network of Show Homes',)

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://0b1ef69e-a1e6-48ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(75, 3)
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to /Users/juliasuter/Documents/repositories/show_homes/show_homes/analysis/maps/Generated_network_map_Detached_House_True_0.01_0.05_30_35_Orkney_Islands.html!
(109, 3)
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to /Users/juliasuter/Documents/repositories/show_homes/show_homes/analysis/maps/Generated_network_map_Terraced_House_False_0.01_0.05_150_35_North_West_Leicestershire.html!


In [19]:
from asf_core_data.getters.supplementary_data.deprivation import imd_data
from asf_core_data.getters.supplementary_data.geospatial import coordinates
from asf_core_data.getters import data_download

from show_homes.config import config


epc_df_with_MCS = pd.read_csv(config.SHOW_HOME_DATA_OUT_DATA_PATH / "epc_df_with_mcs.csv")

# Add IMD data
imd_df = imd_data.get_gb_imd_data()

epc_df = imd_data.merge_imd_with_other_set(
    imd_df, epc_df_with_MCS, postcode_label="POSTCODE"
)

# Add coordinates
coord_df = coordinates.get_postcode_coordinates()
coord_df["POSTCODE"] = coord_df["POSTCODE"].str.replace(" ", "")
epc_df = pd.merge(epc_df, coord_df, on=["POSTCODE"], how="left")

epc_df.loc[
    epc_df["BUILT_FORM"].isin(
        [
            "Enclosed Mid-Terrace",
            "Enclosed End-Terrace",
            "End-Terrace",
            "Mid-Terrace",
        ]
    ),
    "BUILT_FORM",
] = "Terraced"
epc_df = epc_df.loc[~epc_df["LATITUDE"].isna()]

epc_df.to_csv(config.SHOW_HOME_DATA_OUT_DATA_PATH / "epc_for_show_homes.csv")